In [22]:
# import required libraries
import sys
import re
import pandas as pd
import spacy
from pymongo import MongoClient

import gensim
import string
from gensim import corpora
from gensim.corpora.dictionary import Dictionary
from gensim.models.wrappers import LdaMallet
import os

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
import json
import uuid 

In [11]:
# define params for db operations
uri = 'mongodb://localhost:27017/'
database = 'zs_database'
collection_fetch = 'autotags_v2'
collection_push = 'similarities'

# initiate variables
df = pd.DataFrame()
db = object

# connect to db. TODO: Handle exception cases
client = MongoClient(uri)
db = client[database]

# retrieving required data
df = pd.DataFrame(list(db[collection_fetch].find({}, {"_id":0, "lemma_list_without_verbs": 1, "story_id": 1})))

df

,story_id,lemma_list_without_verbs
0,19,"[Radio, ÜberAll, Zukunftsmusik, Mikrofon, Krum..."
1,21,"[Dorf, SauerstoffMathilda, missmutig, Fenster,..."
2,24,"[lang, Witwer, einzig, Tochter, Mutter, Mädche..."
3,25,"[Beste, Party, Buffet, Sinn, Deftigem, Blick, ..."
4,27,"[Endlose, Laubwälder, saftig, Weide, Wildblume..."
...,...,...
195,489,"[Familie, alt, HausErster, Suche, Ausweg, Weiß..."
196,490,"[Wolkenland, Glück, Merkmal, Wolkenlandes, Sky..."
197,491,"[Tag, komplett, Welt, schwarzweiß, Umweltzerst..."
198,492,"[Goldene, Sonnenstrahlen, klaren, Wasseroberfl..."


In [12]:
final_doc = df["lemma_list_without_verbs"]
dictionary = corpora.Dictionary(final_doc)
DT_matrix = [dictionary.doc2bow(doc) for doc in final_doc]

In [21]:
Lda_object = gensim.models.ldamodel.LdaModel
lda_model_1 = Lda_object(DT_matrix, num_topics=20, id2word = dictionary)
#help(Lda_object)
print(json.dumps(lda_model_1.print_topics(num_topics=20, num_words=4),indent=2))


lda_model_1.show_topic(1)

[
  [
    0,
    "0.004*\"Auge\" + 0.004*\"Wald\" + 0.003*\"mein\" + 0.003*\"Kind\""
  ],
  [
    1,
    "0.007*\"Mutter\" + 0.006*\"Auge\" + 0.005*\"Haus\" + 0.005*\"Mama\""
  ],
  [
    2,
    "0.006*\"Vater\" + 0.006*\"Mutter\" + 0.004*\"alt\" + 0.004*\"Eltern\""
  ],
  [
    3,
    "0.004*\"Erde\" + 0.004*\"Vater\" + 0.004*\"Baum\" + 0.004*\"Auge\""
  ],
  [
    4,
    "0.007*\"Baum\" + 0.006*\"Vater\" + 0.005*\"Tier\" + 0.005*\"Auge\""
  ],
  [
    5,
    "0.006*\"Hand\" + 0.005*\"Kopf\" + 0.005*\"Erde\" + 0.005*\"Tier\""
  ],
  [
    6,
    "0.006*\"Hand\" + 0.006*\"Welt\" + 0.005*\"Mutter\" + 0.005*\"Kind\""
  ],
  [
    7,
    "0.005*\"Frau\" + 0.004*\"Auge\" + 0.004*\"Hand\" + 0.004*\"Haus\""
  ],
  [
    8,
    "0.005*\"Frau\" + 0.004*\"Schule\" + 0.004*\"Welt\" + 0.004*\"Auge\""
  ],
  [
    9,
    "0.005*\"Tier\" + 0.005*\"alt\" + 0.005*\"Auge\" + 0.004*\"Frau\""
  ],
  [
    10,
    "0.006*\"Auge\" + 0.004*\"Wasser\" + 0.004*\"Hand\" + 0.004*\"Mama\""
  ],
  [
    11,
    

[('Mutter', 0.00650731),
 ('Auge', 0.005568198),
 ('Haus', 0.0048728804),
 ('Mama', 0.004854219),
 ('Tier', 0.0042372746),
 ('Frau', 0.004012686),
 ('Eltern', 0.0038489972),
 ('Hand', 0.0038143345),
 ('Kind', 0.0037759081),
 ('Vater', 0.003412919)]

In [15]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_1, DT_matrix, dictionary)
pyLDAvis.save_html(vis, 'lda.html')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18     0.008681  0.006322       1        1  12.748207
7      0.006195  0.001761       2        1   9.887595
15    -0.010201 -0.000234       3        1   9.434482
3     -0.002289  0.011761       4        1   9.210918
19     0.001115  0.000667       5        1   6.566332
4     -0.001098  0.003647       6        1   6.140779
1      0.004831  0.006913       7        1   5.836068
12    -0.002247  0.001622       8        1   5.319019
8     -0.001416  0.003473       9        1   4.946195
11     0.006426 -0.000337      10        1   4.469755
16    -0.004898 -0.006550      11        1   4.397918
10     0.005875  0.004413      12        1   4.002644
2     -0.007437  0.005150      13        1   3.603310
0      0.019067 -0.009430      14        1   3.363061
17    -0.003930  0.003004      15        1   3.125048
6     -0.002283 -0.001718      16        1   3.090286
14    -0.007729 -0.003871      17        1   1.857527
5     -0.002503 -0.008133      18        1   1.433568
13    -0.003120 -0.011584      19        1   0.560773
9     -0.003040 -0.006877      20        1   0.006518, topic_info=        Term        Freq       Total Category  logprob  loglift
96      Erde  753.000000  753.000000  Default  30.0000  30.0000
95    Eltern  675.000000  675.000000  Default  29.0000  29.0000
394     Welt  660.000000  660.000000  Default  28.0000  28.0000
739     Frau  695.000000  695.000000  Default  27.0000  27.0000
64      Baum  786.000000  786.000000  Default  26.0000  26.0000
...      ...         ...         ...      ...      ...      ...
185     Kopf    0.033655  687.076782  Topic20  -5.9657  -0.2856
156     Hand    0.037007  835.585815  Topic20  -5.8707  -0.3864
1128  nächst    0.028770  519.514038  Topic20  -6.1225  -0.1629
219     Luft    0.026906  459.598511  Topic20  -6.1895  -0.1073
741   Freund    0.025074  478.766052  Topic20  -6.2600  -0.2187

[1620 rows x 6 columns], token_table=       Topic      Freq Term
term                       
13973      1  0.063029    <
13973      3  0.189087    <
13973      4  0.283631    <
13973      5  0.220602    <
13973      8  0.031515    <
...      ...       ...  ...
12862      8  0.060489    €
12862      9  0.060489    €
12862     12  0.060489    €
12862     13  0.060489    €
12862     17  0.060489    €

[6234 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 8, 16, 4, 20, 5, 2, 13, 9, 12, 17, 11, 3, 1, 18, 7, 15, 6, 14, 10])

In [38]:
def topic_modeling_viz(num_topics=5):
    uri = 'mongodb://localhost:27017/'
    database = 'zs_database'
    collection_fetch = 'autotags_v2'
    collection_push = 'similarities'

    # initiate variables
    df = pd.DataFrame()
    db = object

    # connect to db. TODO: Handle exception cases
    client = MongoClient(uri)
    db = client[database]

    # retrieving required data
    df = pd.DataFrame(list(db[collection_fetch].find({}, {"_id":0, "lemma_list_without_verbs": 1, "story_id": 1})))

    final_doc = df["lemma_list_without_verbs"]
    dictionary = corpora.Dictionary(final_doc)
    DT_matrix = [dictionary.doc2bow(doc) for doc in final_doc]
    Lda_object = gensim.models.ldamodel.LdaModel
    lda_model_1 = Lda_object(DT_matrix, num_topics=num_topics, id2word = dictionary)
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim.prepare(lda_model_1, DT_matrix, dictionary)
    filename = f"../templates/lda_{num_topics}_{uuid.uuid1()}.html"
    pyLDAvis.save_html(vis, filename)
    
def topic_modeling_topic(num_topics,topic_id):
    uri = 'mongodb://localhost:27017/'
    database = 'zs_database'
    collection_fetch = 'autotags_v2'
    collection_push = 'similarities'

    # initiate variables
    df = pd.DataFrame()
    db = object

    # connect to db. TODO: Handle exception cases
    client = MongoClient(uri)
    db = client[database]

    # retrieving required data
    df = pd.DataFrame(list(db[collection_fetch].find({}, {"_id":0, "lemma_list_without_verbs": 1, "story_id": 1})))

    final_doc = df["lemma_list_without_verbs"]
    dictionary = corpora.Dictionary(final_doc)
    DT_matrix = [dictionary.doc2bow(doc) for doc in final_doc]
    Lda_object = gensim.models.ldamodel.LdaModel
    lda_model_1 = Lda_object(DT_matrix, num_topics=num_topics, id2word = dictionary)
    topics = lda_model_1.show_topic(topic_id)
    return topics 
    

In [39]:
topic_modeling_viz(2)

In [59]:
import numpy as np
topics = topic_modeling_topic(2,1)


0 Baum 0.004333070013672113
<class 'numpy.float32'>
1 Erde 0.004175593610852957
<class 'numpy.float32'>
2 Auge 0.004173164255917072
<class 'numpy.float32'>
3 Haus 0.004037362057715654
<class 'numpy.float32'>
4 Frau 0.0038442309014499187
<class 'numpy.float32'>
5 Hand 0.003716830164194107
<class 'numpy.float32'>
6 Tier 0.003479441162198782
<class 'numpy.float32'>
7 alt 0.003463607979938388
<class 'numpy.float32'>
8 Vater 0.0033370035234838724
<class 'numpy.float32'>
9 mein 0.0033049487974494696
<class 'numpy.float32'>


'[{"word": "Baum", "prob": 0.004333070013672113}, {"word": "Erde", "prob": 0.004175593610852957}, {"word": "Auge", "prob": 0.004173164255917072}, {"word": "Haus", "prob": 0.004037362057715654}, {"word": "Frau", "prob": 0.0038442309014499187}, {"word": "Hand", "prob": 0.003716830164194107}, {"word": "Tier", "prob": 0.003479441162198782}, {"word": "alt", "prob": 0.003463607979938388}, {"word": "Vater", "prob": 0.0033370035234838724}, {"word": "mein", "prob": 0.0033049487974494696}]'